In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_blobs
from and_logic_generator import get_not_y

In [ ]:
# we create 40 separable points
random_state = 2**3
x, y = make_blobs(n_samples=40, centers=2, random_state=random_state)

In [ ]:
# fit the model, don't regularize for illustration purposes
model = svm.SVC(kernel='linear', C=100)
model.fit(x, y)

In [ ]:
x0_mean = x[:, 0].mean()
x0_min = x[:, 0].min() - x0_mean * 0.5
x0_max = x[:, 0].max() + x0_mean * 0.5
x1_mean = x[:, 1].mean()
x1_min = x[:, 1].min() - x1_mean * 0.5
x1_max = x[:, 1].max() + x1_mean * 0.5

In [ ]:
w = model.coef_[0]
print('coef_', w)
a = -w[0] / w[1]
xx = np.linspace(x0_min, x0_max)
yy = a * xx - (model.intercept_[0]) / w[1]
print('intercept_', model.intercept_)
print(xx)
print(yy)

margin = 1 / np.sqrt(np.sum(model.coef_ ** 2))
yy_down = yy - np.sqrt(1 + a ** 2) * margin
yy_up = yy + np.sqrt(1 + a ** 2) * margin

In [ ]:
# creating adversarial examples on SVM with linear kernal
x_test = np.array([[6.0, 1.0], [8.0, 2.0], [10.0, 10.0]], dtype=np.float32)
pred_test = model.predict(x_test)
print(pred_test)

sup_x = model.support_vectors_
dual_alpha = model.dual_coef_
beta = model.intercept_

# coef_ is only computed in linear kernel
# w = model.coef_
w = dual_alpha @ sup_x
w_l2_norm = np.sqrt(np.sum(w**2))
w_norm = w / w_l2_norm
print('w_norm:\n', w_norm)

print(x_test.shape)
print(w.shape)
Q = (x_test @ w.T + beta)
print('Q:\n', Q)

# y in set of {1, -1}. Transform label 0 to -1.
y_sign = np.array([[-1 if yy == 0 else 1 for yy in pred_test]], dtype=np.float32).T
print('y_sign:\n', y_sign)

epsilon = 1.0
delta = - w_norm * (Q + y_sign * (margin + epsilon))
adversarial = x_test + delta
print('adversarial:\n', adversarial)


In [ ]:
def get_gradient_linear(support_x, dual_alpha):
    """
    The gradient in linear kernel does not vary based on x
    """
    return np.sum(dual_alpha.T * support_x, axis=0)

In [ ]:
# Using an interative algorithm to find the minimal vector
learning_rate = 0.8
x_star = np.array(x_test)
y_not = get_not_y(pred_test)
pred_x_star = model.predict(x_star)
inds_update = np.where(pred_x_star != y_not)[0]

x_stack = np.array([], dtype=np.float32)

epoch = 0
max_epoches = 20
while len(inds_update) > 0 and epoch < max_epoches:
    x_update = x_star[inds_update] - y_sign[inds_update] * learning_rate * get_gradient_linear(sup_x, dual_alpha)      
    x_star[inds_update] = x_update

    pred_x_star = model.predict(x_star)
    inds_update = np.where(pred_x_star != y_not)[0]
    epoch = epoch + 1
    
    if len(x_stack) == 0:
        x_stack = x_update
    else:
        x_stack = np.vstack((x_stack, x_update))

print('before df = ', model.decision_function(x_test))
print(f'after {epoch} epochs df = ', model.decision_function(x_star))

In [ ]:
XX, YY = np.mgrid[x0_min:x0_max:200j, x1_min:x1_max:200j]
Z = model.predict(np.c_[XX.ravel(), YY.ravel()])
Z = Z.reshape(XX.shape)

figsize = np.array(plt.rcParams["figure.figsize"]) * 2
plt.figure(figsize=figsize.tolist())

plt.contourf(XX, YY, Z, cmap='coolwarm', alpha=0.4, zorder=1)

plt.plot(xx, yy, 'k-')
plt.plot(xx, yy_down, 'k--')
plt.plot(xx, yy_up, 'k--')

plt.scatter(x[:, 0], x[:, 1], c=y, cmap='coolwarm', edgecolor='face', s=40, zorder=2)
plt.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1], c=y[model.support_], cmap='coolwarm', edgecolors='white', linewidths=2, s=80, zorder=10)

pred_test = model.predict(x_test)
plt.scatter(x_test[:, 0], x_test[:, 1], c=pred_test, cmap='coolwarm', edgecolor='k', s=80, zorder=20)
plt.scatter(adversarial[:, 0], adversarial[:, 1], c=model.predict(adversarial), cmap='coolwarm', edgecolor='lime', s=80, zorder=20)

pred_iter_adversarials = model.predict(x_star)
plt.scatter(x_star[:, 0], x_star[:, 1], c=pred_iter_adversarials, cmap='coolwarm', edgecolor='red', s=80, zorder=20)

plt.scatter(x_stack[:, 0], x_stack[:, 1], c=model.predict(x_stack), cmap='coolwarm', edgecolor='k', s=80, zorder=10, alpha=0.4)

plt.xlim(x0_min, x0_max)
plt.ylim(x1_min, x1_max)
plt.show()

## Some links
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

https://scikit-learn.org/stable/modules/svm.html

https://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane.html

https://scikit-learn.org/stable/auto_examples/svm/plot_svm_margin.html

https://scikit-learn.org/stable/auto_examples/exercises/plot_iris_exercise.html

https://towardsdatascience.com/support-vector-machine-python-example-d67d9b63f1c8

https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html
